In [14]:
import dask
import dask.array as da
from skimage import feature
from PIL import Image
import numpy as np
import time
import matplotlib.pyplot as plt

Image.MAX_IMAGE_PIXELS = None

In [15]:
# Set configurations for Dask
dask.config.set(
    scheduler='threads',
    num_workers=4,
    memory_limit='48GB',  # 80% of 64GB RAM
    temporaries=True
)

In [16]:
def calculate_lbp(image, radius, points):
    # Convert to NumPy array if necessary
    image_np = np.array(image)

    # Your code to calculate LBP
    lbp_result = feature.local_binary_pattern(image_np, points, radius, method='uniform')
    return lbp_result

In [17]:
def calculate_lbp_delayed(image_path, radius, points):
    # Open the image using dask.array.from_array with adjusted chunk size
    image = da.from_array(np.array(Image.open(image_path)), chunks=(1000, 1000))

    # Calculate LBP with the specified parameters
    result = calculate_lbp(image, radius, points)
    return result

In [18]:
def parallel_lbp(image_path, radius_list, points_list):
    # Create tasks to calculate LBP for different parameters
    delayed_tasks = [dask.delayed(calculate_lbp_delayed)(image_path, radius, points) for radius in radius_list for points in points_list]

    # Gather results
    start_time = time.time()
    results = dask.compute(*delayed_tasks, num_workers=len(delayed_tasks))
    end_time = time.time()

    # Print execution time
    print(f"Execution time: {end_time - start_time} seconds")

    return results

In [19]:
def plot_results(results, radius_list, points_list):
    num_results = len(results)
    
    fig, axes = plt.subplots(len(radius_list), len(points_list), figsize=(12, 12))

    for i, radius in enumerate(radius_list):
        for j, points in enumerate(points_list):
            index = i * len(points_list) + j
            ax = axes[i, j]

            ax.imshow(results[index], cmap='gray')
            ax.set_title(f"Radius={radius}, Points={points}")
            ax.axis('off')

    plt.tight_layout()
    plt.show(block=False)


In [20]:
# Example usage
image_path = '/lustre/scratch126/casm/team268im/yk3/testDask&Cupy/img_L_10000x10000.tiff'
radius_list = [1, 2, 3]
points_list = [8, 16]

# Call the parallel function
results = parallel_lbp(image_path, radius_list, points_list)

# Visualize the results
# plot_results(results, radius_list, points_list)

Execution time: 15.672741174697876 seconds
